In [1]:
import ipywidgets as widgets
from IPython.display import display, Markdown, clear_output
import pandas as pd
import numpy as np
import copy
import sympy as sp
import ipysheet
from ipysheet import column, row, sheet, cell_range, sheet, cell  
import simplex_algorithm  as sa
import tableau_input2 as ti
import functools
from graphviz import Graph

In [2]:
M = sp.symbols('M')
max_simplex_iterations = 10 #number of iterations before simplex stops
max_iterations = 10 #max branches in tree
max_real_vars = 10

In [3]:
def create_input_table_with_file(change, display_table_input, display_output, M):
    # clear old output
    display_output.clear_output()
    display_table_input.clear_output()
    
    #define widgets
    button_start_branch_and_bound = widgets.Button(description="Starte Branch and Bound", disabled=True)
    button_check_input = widgets.Button(description="Überprüfe Eingabe")
    select_positive_integer_var = widgets.SelectMultiple(description='Select integer Var >= 0')
    select_binary_var = widgets.SelectMultiple(description='Select binary Var')
    
    uploaded_tableau, integer_vars, binary_vars = read_upload_variables(change)
      
    #get count of real variables by number of variables with x
    count_x_var = 0
    for var_name in uploaded_tableau[1][2:]:
        if "x" in var_name:
            count_x_var +=1
            
    #fill missing Attributes from Multiple-Widgets       
    select_positive_integer_var.options= ["x" + str(i) for i in range(1, count_x_var+1)]
    select_positive_integer_var.value = integer_vars
    select_binary_var.options= ["x" + str(i) for i in range(1, count_x_var+1)]
    select_binary_var.value = binary_vars
     
    # create input table with variable from file
    input_table = sheet(rows=len(uploaded_tableau), columns=len(uploaded_tableau[0]),
                        row_headers=False, column_headers=False)
    cell_table = fill_and_observe_cells(button_start_branch_and_bound, input_table, uploaded_tableau)

    #display
    with display_table_input:
        display(Markdown("## Laden des Standardtableaus"))
        display(input_table)
        display(select_positive_integer_var)
        display(select_binary_var)
        display(button_check_input)
        display(button_start_branch_and_bound)
    
            
        
    button_check_input.on_click(functools.partial(check_binary_or_integer,
                                                  select_binary_var=select_binary_var,
                                                  select_positive_integer_var=select_positive_integer_var))
    
    button_check_input.on_click(functools.partial(ti.check_correct_input,
                                                  button_start_algorithm=button_start_branch_and_bound,
                                                  cell_table=cell_table, input_table=input_table))
    
    button_start_branch_and_bound.on_click(functools.partial(start_branch_and_bound, input_table = input_table, 
                                                             sum_var = input_table.columns-3, 
                                                             select_positive_integer_var = select_positive_integer_var, 
                                                             select_binary_var = select_binary_var, 
                                                             display_output = display_output, M=M))                                            
    
    
    
    
def create_input_table_manually(button, dd_count_restr, dd_count_x_vars ,dd_count_s_vars, 
                                dd_count_a_vars ,display_table_input, display_output,M): 
    # clear old output
    display_output.clear_output()
    display_table_input.clear_output()
    
    #define widgets
    button_start_branch_and_bound = widgets.Button(description="Starte Branch and Bound", disabled=True)
    button_check_input = widgets.Button(description="Überprüfe Eingabe")
    select_positive_integer_var = widgets.SelectMultiple(options=["x" + str(i) for i in range(1, int(dd_count_x_vars.value)+1)],
                                                         value=["x" + str(i) for i in range(1, int(dd_count_x_vars.value)+1)],
                                                         description='Select integer Var >= 0')
    select_binary_var = widgets.SelectMultiple(options=["x" + str(i) for i in range(1, int(dd_count_x_vars.value)+1)], 
                                               value=["x" + str(i) for i in range(1, int(dd_count_x_vars.value)+1)],
                                               description='Select binary Var')


    #create input table
    sum_var=int(dd_count_x_vars.value) + int(dd_count_s_vars.value) + int(dd_count_a_vars.value)
    input_table = ipysheet.sheet(rows=2+float(dd_count_restr.value), columns=sum_var+3,  
                                 row_headers=False, column_headers=False)
    input_table, cell_table = ti.prefill_tableau_text_and_colour(input_table=input_table,
                                                                 dd_count_restr=dd_count_restr, 
                                                                 dd_count_x_vars=dd_count_x_vars,
                                                                 dd_count_s_vars=dd_count_s_vars, 
                                                                 dd_count_a_vars=dd_count_a_vars,
                                                                 button_start_algorithm=button_start_branch_and_bound,
                                                                 M=M)
    
    #display
    with display_table_input:
        display(Markdown("## Erzeuge das Standardtableau"))
        display(input_table)
        display(select_positive_integer_var)
        display(select_binary_var)
        display(button_check_input)
        display(button_start_branch_and_bound)
        
   
    #observe/on_click-function
    button_check_input.on_click(functools.partial(check_binary_or_integer,
                                                  select_binary_var=select_binary_var,
                                                  select_positive_integer_var=select_positive_integer_var))
    #observe/on-click-functions                                                                       
    button_check_input.on_click(functools.partial(ti.check_correct_input,
                                                  button_start_algorithm=button_start_branch_and_bound, 
                                                  cell_table=cell_table,
                                                  input_table=input_table))
    
    button_start_branch_and_bound.on_click(functools.partial(start_branch_and_bound, input_table=input_table, 
                                                             sum_var=sum_var, select_positive_integer_var=select_positive_integer_var, 
                                                             select_binary_var=select_binary_var, display_output=display_output, M=M))


    
    
def start_branch_and_bound(button, input_table, sum_var, select_positive_integer_var,
                           select_binary_var, display_output, M):
    #variables
    info_dots = [] #info for edges
    node_name_counter = 0
    correct_solutions = [] 
    open_dots = [] #possible dot to make branches
    
    dot = Graph()  # dot in binary tree
    
    # clear old input
    display_output.clear_output()

    #adjust input table to input format for simplex-algorithm
    df_tableau = adjust_input_table_for_simplex(input_table, select_binary_var,
                                                select_positive_integer_var, sum_var, M)

    # use simplex-algorithm on tableau
    dfs_tableau, messages, pivot_elements_ids = sa.simplex_algorithm(df_tableau, max_simplex_iterations, M)

    with display_output:
        display(Markdown("## Ergebnis"))

    # create start node
    dot, open_dots, node_name_counter, \
    correct_solutions, info_dots = create_node(dot, dfs_tableau[0], dfs_tableau[-1], messages,
                                               open_dots, node_name_counter, correct_solutions,
                                               select_positive_integer_var, select_binary_var, info_dots)

    counter_loop = 0
    while len(open_dots) != 0:
        # select dot to extend and create the two needed talbeaus
        selected_open_dot = open_dots[0]
        standard_tableau_left, standard_tableau_right = create_left_right_tableau(selected_open_dot, M)

        # create dot and edge for left branch
        correct_solutions, dot, info_dots, \
        node_name_counter, open_dots = create_left_branch(M, correct_solutions, dot, info_dots,
                                                          node_name_counter, open_dots, select_binary_var, 
                                                          select_positive_integer_var, selected_open_dot,
                                                          standard_tableau_left)

        # create dot and edge for right branch
        correct_solutions, dot, info_dots, \
        node_name_counter, open_dots = create_right_branch(M, correct_solutions, dot, info_dots,
                                                           node_name_counter, open_dots, select_binary_var,
                                                           select_positive_integer_var, selected_open_dot, 
                                                           standard_tableau_right)

        # update lists
        open_dots.pop(0)
        open_dots = sorted(open_dots)
        correct_solutions = sorted(correct_solutions)

        # leave loop when no better solution can be found
        if len(open_dots) >= 1 and len(correct_solutions) >= 1 \
                and open_dots[0][0] <= correct_solutions[0][ 0]:
            break

        # prevent continuous loop
        counter_loop += 1
        if counter_loop == max_iterations:
            break

    # create dataframe of the dot informations
    df_node_info = pd.DataFrame(info_dots, columns=["Node Name", "Values"])

    # display
    with display_output:
        display(dot)
        display(df_node_info.style.hide_index())


def create_right_branch(M, correct_solutions, dot, info_dots, node_name_counter, open_dots, select_binary_var,
                        select_positive_integer_var, selected_open_dot, standard_tableau_right):
    #simplex-algorithm on new tableau
    dfs_tableau, messages, pivot_elements_ids = sa.simplex_algorithm(standard_tableau_right,
                                                                     max_simplex_iterations, M)
    final_tableau_right = dfs_tableau[-1]
    
    #create dot depending on simplex result
    dot, open_dots, node_name_counter, \
    correct_solutions, info_dots = create_node(dot, standard_tableau_right, final_tableau_right,
                                               messages, open_dots, node_name_counter, correct_solutions,
                                               select_positive_integer_var, select_binary_var, info_dots)
    
    #create edge to dot 
    dot.edge(selected_open_dot[1], 'P' + str(node_name_counter - 1),
             label=" " + selected_open_dot[3][0] + "≥ " + str(int(selected_open_dot[3][1] + 1)))
    return correct_solutions, dot, info_dots, node_name_counter, open_dots


def create_left_branch(M, correct_solutions, dot, info_dots, node_name_counter, open_dots, select_binary_var,
                       select_positive_integer_var, selected_open_dot, standard_tableau_left):
    #simplex-algorithm on new tableau
    dfs_tableau, messages, pivot_elements_ids = sa.simplex_algorithm(standard_tableau_left,
                                                                     max_simplex_iterations, M)
    final_tableau_left = dfs_tableau[-1]
    
    #create dot depending on simplex result
    dot, open_dots, node_name_counter, \
    correct_solutions, info_dots = create_node(dot, standard_tableau_left, final_tableau_left,
                                               messages, open_dots, node_name_counter, correct_solutions,
                                               select_positive_integer_var, select_binary_var, info_dots)
    
    #create edge to dot
    dot.edge(selected_open_dot[1], 'P' + str(node_name_counter - 1),
             xlabel=" " + selected_open_dot[3][0] + "≤ " + str(int(selected_open_dot[3][1])))
    return correct_solutions, dot, info_dots, node_name_counter, open_dots

def adjust_input_table_for_simplex(input_table, select_binary_var, select_positive_integer_var, sum_var, M):
    # make the tableau a df
    df_tableau = ipysheet.to_dataframe(input_table)
    df_tableau = df_tableau.apply(pd.to_numeric, errors='ignore', downcast='float')
    df_tableau.columns = range(0, sum_var + 3)

    # variables cant be binary and a positiv integer
    temp_list = list(select_positive_integer_var.value)
    for var in temp_list:
        if var in select_binary_var.value:
            temp_list.remove(var)
    select_positive_integer_var.value = temp_list

    # add restriction so the binary var is 0 or 1
    df_tableau = add_restr_for_binary_vars(df_tableau, select_binary_var)

    # add cj and cj-zj row
    df_tableau = ti.add_missing_cj_and_cj_zj_rows(df_tableau, M)
    df_tableau = sa.calculate_cj_and_cj_zj(df_tableau)

    return df_tableau


def add_restr_for_binary_vars(df_tableau, select_binary_var):
    for binary_var in select_binary_var.value:
        count_x_var = sum(df_tableau.iloc[1, 3:].str.count("x"))
        count_s_var = sum(df_tableau.iloc[1, 3:].str.count("s"))
        new_column_id = 2 + count_x_var + count_s_var + 1
        binary_var_column_id = int(binary_var[1:]) + 2

        #create column for restriction
        new_column = []
        for i in range(0, len(df_tableau.index)):
            if i == 1:
                new_column.append("s" + str(count_s_var + 1))
            else:
                new_column.append(0)

        #add column to df_tableau
        df_tableau.insert(2 + count_x_var + count_s_var + 1, "new_column", new_column)

        #create row for new restriction variable 
        new_row = []
        for i in range(0, len(df_tableau.columns)):
            if i == 2 or i == new_column_id or i == binary_var_column_id:
                new_row.append(1)
            elif i == 1:
                new_row.append("s" + str(count_s_var + 1))
            else:
                new_row.append(0)

        #add row to df_tableau and remake index
        df_tableau = pd.DataFrame(np.insert(df_tableau.values, len(df_tableau) - 2, values=new_row, axis=0))
        df_tableau.index = range(0, len(df_tableau.index))
    return df_tableau

       
    
def create_node(tree, standard_tableau, final_tableau, messages, open_dots, node_name_counter, correct_solutions,
                select_positive_integer_var, select_binary_var, info_dots):
    
    #get info of dots for dataframe
    final_value, is_right_formats, selected_next_var = get_info_to_dots(final_tableau, info_dots, messages,
                                                                           node_name_counter, select_binary_var,
                                                                           select_positive_integer_var)

    select_shape_and_draw_dot(correct_solutions, final_value, info_dots, is_right_formats, messages,
                              node_name_counter, open_dots, selected_next_var, standard_tableau, tree)

    return tree, open_dots, node_name_counter, correct_solutions, info_dots


def get_info_to_dots(final_tableau, info_dots, messages, node_name_counter, select_binary_var,
                     select_positive_integer_var):
    #variables
    selected_next_var = [None, None]  # var_name and var_quantity
    values = ""
    is_right_formats = True  # all variables have the intended format (float, integer, binary)
    result_x_vars = []  # [["x1" , final-value in optimized tableau],...]
    
    # adjust decimal places of final value result
    final_value = final_tableau.iloc[-2, 2]
    if isinstance(final_value, float) and final_value != int(final_value):
        final_value = "%.2f" % final_value
    elif isinstance(final_value, float) and final_value == int(final_value):
        final_value = "%.0f" % final_value
        
    if len(messages[-1]) == 0:  # no message = no mistake = solved successfully
        
        # get all real variables in basis variables and their quantity
        for row_id in range(0, len(final_tableau)):
            if "x" in final_tableau.iloc[row_id, 1]:
                result_x_vars.append([final_tableau.iloc[row_id, 1], round(final_tableau.iloc[row_id, 2], 10)])

        # adjust decimal digits and check for right format of real variables
        for elements in sorted(result_x_vars):
            values = values + elements[0] + "="

            # check for integer (all except binary variables)
            if elements[1] == int(elements[1]) and elements[0] not in select_binary_var.value:
                values = values + str("%.0f" % elements[1])
            elif isinstance(elements[1], float) and elements[1] != int(elements[1]) and \
                    elements[0] in select_positive_integer_var.value:
                is_right_formats = False
                values = values + str("%.2f" % elements[1])
                if selected_next_var == [None, None]:
                    selected_next_var = elements

            # check for float variables
            elif elements[0] not in select_positive_integer_var.value and elements[1] != int(elements[1]):
                values = values + str("%.2f" % elements[1])

            # check binary condition
            elif elements[1] == 0 or elements[1] == 1 and elements[0] in select_binary_var.value:
                values = values + str("%.0f" % elements[1])
            elif elements[1] != 0 and elements[1] != 1 and elements[0] in select_binary_var.value:
                is_right_formats = False
                values = values + str("%.2f" % elements[1])
                selected_next_var = [elements[0], 0.5]

            if elements != sorted(result_x_vars)[-1]:
                values = values + ", "
            else:
                values = "(" + values + ")" + " Z=" + str(final_value)

        info_dots.append([('P' + str(node_name_counter - 1)), values])
    return final_value, is_right_formats, selected_next_var


def select_shape_and_draw_dot(correct_solutions, final_value, info_dots, is_right_formats, messages,
                              node_name_counter, open_dots, selected_next_var, standard_tableau, tree):
    #if solution was found
    if len(messages[-1]) == 0:
        if is_right_formats == True: #correct solution
            tree.attr('node', shape='doublecircle')
            tree.node('P' + str(node_name_counter))
            node_name_counter = node_name_counter + 1
            correct_solutions.append([final_value, 'P' + str(node_name_counter)])
        elif is_right_formats == False: #not all criteria were met
            tree.attr('node', shape='circle')
            tree.node('P' + str(node_name_counter))

            #if dot has possibly better solutions, add to open_dots
            if len(correct_solutions) == 0 or \
                    (len(correct_solutions) != 0 and correct_solutions[-1][0] < final_value):
                open_dots.append([final_value, "P" + str(node_name_counter), standard_tableau, selected_next_var])

            elif len(correct_solutions) != 0 and correct_solutions[-1][0] > final_value:
                tree.attr('node', shape='circle')
                tree.node('P' + str(node_name_counter))
                node_name_counter = node_name_counter + 1

            node_name_counter = node_name_counter + 1

    #if no solution was found
    elif len(messages[-1]) != 0:
        for error in messages[-1]:
            if "Unausführbarkeit" in error:
                tree.attr('node', shape='square')
                tree.node('P' + str(node_name_counter))
                node_name_counter = node_name_counter + 1
                info_dots.append([('P' + str(node_name_counter - 1)), "Unausführbarkeit"])
            elif "unbeschränkter Lösungsraum" in error:
                tree.attr('node', shape='diamond')
                tree.node('P' + str(node_name_counter))
                node_name_counter = node_name_counter + 1
                info_dots.append([('P' + str(node_name_counter - 1)), "unbeschränkter Lösungsraum"])
        
def create_left_right_tableau(selected_open_dot, M):
    # variables
    selected_open_dot = selected_open_dot.copy()
    new_restr_left = int(selected_open_dot[3][1])
    new_restr_right = int(selected_open_dot[3][1]) + 1
    var_name = selected_open_dot[3][0]

    count_x_var = sum(selected_open_dot[2].iloc[1, 3:].str.count("x"))
    count_s_var = sum(selected_open_dot[2].iloc[1, 3:].str.count("s"))
    count_a_var = sum(selected_open_dot[2].iloc[1, 3:].str.count("a"))


    selected_tableau = selected_open_dot[2].copy(deep=True)

    # adjust tableau to restriction for x <= new_upper_restr
    standard_tableau_left = add_restr_for_left_dot(count_s_var, count_x_var, new_restr_left,
                                                   selected_tableau, var_name)

    # adjust tableau to restriction for x >= new_lower_restr
    standard_tableau_right = add_restr_for_right_dot(M, count_a_var, count_s_var, count_x_var, new_restr_right,
                                                     selected_tableau, var_name)

    return standard_tableau_left, standard_tableau_right
    


def add_restr_for_right_dot(M, count_a_var, count_s_var, count_x_var, 
                            new_restr_right, selected_tableau, var_name):
    #copy of selected dot tableau
    standard_tableau_right = selected_tableau.copy(deep=True)
    
    # create new columns for right dot (x_ <= value)
    new_column_1 = []
    for i in range(0, len(standard_tableau_right)):
        if i == 1:
            new_column_1.append("s" + str(count_s_var + 1))
        else:
            new_column_1.append(0)
            
    new_column_2 = []
    for i in range(0, len(standard_tableau_right)):
        if i == 0 or i == len(standard_tableau_right) - 2:
            new_column_2.append(-M)
        elif i == 1:
            new_column_2.append("a" + str(count_a_var + 1))
        else:
            new_column_2.append(0)
            
    # add columns to tableau
    standard_tableau_right.insert(2 + count_x_var + count_s_var + 1, "new_column1", new_column_1)
    standard_tableau_right.insert(len(standard_tableau_right.columns), "new_column2", new_column_2)
    standard_tableau_right.columns = range(0, len(standard_tableau_right.columns))
    
    # create new row for right dot (x_ <= value)
    new_row = []
    for i in range(0, len(standard_tableau_right.columns)):
        if i == 0:
            new_row.append(-M)
        elif i == 1:
            new_row.append("a" + str(count_a_var + 1))
        elif i == 2:
            new_row.append(new_restr_right)
        elif i == int(var_name[1:]) + 2:
            new_row.append(1)
        elif i == 2 + count_x_var + count_s_var + 1:
            new_row.append(-1)
        elif i == len(standard_tableau_right.columns) - 1:
            new_row.append(1)
        else:
            new_row.append(0)
            
    # add row to tableau
    standard_tableau_right = pd.DataFrame(np.insert(standard_tableau_right.values,
                                                    len(standard_tableau_right) - 3, values=new_row, axis=0))
    standard_tableau_right.index = range(0, len(standard_tableau_right.index))
    
    return standard_tableau_right


def add_restr_for_left_dot(count_s_var, count_x_var, new_restr_left, selected_tableau, var_name):
    
    #copy of selected dot tableau
    standard_tableau_left = selected_tableau.copy(deep=True)
    
    # create new column for left dot (x_ <= value)
    new_column = []
    for i in range(0, len(standard_tableau_left)):
        if i == 1:
            new_column.append("s" + str(count_s_var + 1))
        else:
            new_column.append(0)
            
    # add column to tableau
    standard_tableau_left.insert(2 + count_x_var + count_s_var + 1, "new_column", new_column)
    standard_tableau_left.columns = range(0, len(standard_tableau_left.columns))
    
    # create new row for left dot (x_ <= value)
    new_row = []
    for i in range(0, len(standard_tableau_left.columns)):
        if i == 1:
            new_row.append("s" + str(count_s_var + 1))
        elif i == 2:
            new_row.append(new_restr_left)
        elif i == int(var_name[1:]) + 2:
            new_row.append(1)
        elif i == 2 + count_x_var + count_s_var + 1:
            new_row.append(1)
        else:
            new_row.append(0)
            
    # add row to tableau
    standard_tableau_left = pd.DataFrame(np.insert(standard_tableau_left.values, 
                                                   len(standard_tableau_left) - 2, values=new_row, axis=0))
    
    return standard_tableau_left

In [4]:
def create_dropdowns():
    # quantity of restrictions
    dd_count_restr = widgets.Dropdown(
        options=list(map(str, range(0, 6))),
        value='0',
        description='Restriktionen:',
        disabled=False,
    )

    # quantity of real variables
    dd_count_x_vars = widgets.Dropdown(
        options=list(map(str, range(1, max_real_vars))),
        value='1',
        description='echte Variablen:',
        disabled=False,
    )

    # quantity of slack variables
    dd_count_s_vars = widgets.Dropdown(
        options=list(map(str, range(0, 6))),
        value='0',
        description='Schlupfvariablen:',
        disabled=False,
    )

    # quantity of artificial variables
    dd_count_a_vars = widgets.Dropdown(
        options=list(map(str, range(0, 6))),
        value='0',
        description='künstliche Variablen:',
        disabled=False,
    )
    return dd_count_restr, dd_count_x_vars, dd_count_s_vars, dd_count_a_vars

def read_upload_variables(change):
    for key in change["new"]:
        string = change["new"][key]['content'].decode("utf-8")
        var_to_value = {}
        exec(string, var_to_value)
        uploaded_tableau = var_to_value['tableau']
        integer_vars = var_to_value['integer_variables']
        binary_vars = var_to_value['binary_variables']
    return uploaded_tableau, integer_vars, binary_vars

def fill_and_observe_cells(button_simplex_start, input_table, uploaded_tableau):
    #look at every value of the uploaded tableau
    cell_table = [[''] * input_table.columns for i in range(input_table.rows)]
    for row_id in range(input_table.rows):
        for column_id in range(input_table.columns):
            value = uploaded_tableau[row_id][column_id]
            
            #define cell font_weight
            if row_id == 1 or column_id == 1:
                font_weight = 'bold'
            else:
                font_weight = None
            
            #define cell color
            if row_id == 0 and (column_id == 0 or column_id == 1 or column_id == 2) and value != '0':
                cell_color = 'grey'
            elif (row_id == 0 or column_id == 0) and (value == '-M' or value == 0):
                cell_color = 'white'
            elif row_id == 1 or column_id == 1:
                cell_color = 'white'
            else:
                cell_color = 'yellow'
            
            #give value to cell with defined attributes
            cell_table[row_id][column_id] = cell(row_id, column_id, value, background_color=cell_color,
                                                 numeric_format='0.', font_weight=font_weight)
            #observe every cell for changes
            cell_table[row_id][column_id].observe(functools.partial(ti.update_table,
                                                                    input_table=input_table,
                                                                    button_start_algorithm=button_simplex_start))
    return cell_table

def check_binary_or_integer(button, select_binary_var, select_positive_integer_var):
        for var_name in select_binary_var.value:
            if var_name in select_positive_integer_var.value:
                temp_list = list(select_positive_integer_var.value)
                temp_list.remove(var_name)
                select_positive_integer_var.value = temp_list

In [5]:
# create widgets
display_variable_input = widgets.Output()
display_table_input = widgets.Output()
display_output = widgets.Output()
uploader = widgets.FileUpload(accept='.txt', multiple=False)


dd_count_restr, dd_count_x_vars, dd_count_s_vars, dd_count_a_vars = create_dropdowns()

button_create_input_table = widgets.Button(description='Erzeuge Eingabetableau!')

# manage widget visibility
dd_count_s_vars.layout.visibility = 'hidden'
dd_count_a_vars.layout.visibility = 'hidden'
button_create_input_table.layout.visibility = 'hidden'

# display
display(Markdown('# Branch and Bound'))
display(Markdown('## Definiere Art/Anzahl der Variablen'))
display(display_variable_input)
display(display_table_input)
display(display_output)

with display_variable_input:
    display(uploader)
    display(Markdown('#### oder'))
    display(dd_count_restr)
    display(dd_count_x_vars)
    display(dd_count_s_vars)
    display(dd_count_a_vars)
    display(button_create_input_table)

# adjust dropdown values by count restriction change
dd_count_restr.observe(functools.partial(ti.on_restr_change,
                                         dd_count_x_vars=dd_count_x_vars,
                                         dd_count_s_vars=dd_count_s_vars,
                                         dd_count_a_vars=dd_count_a_vars,
                                         button_create_input_table=button_create_input_table,
                                         display_variable_input=display_variable_input),
                       names='value'
                       )

# create a table with the values of the dropdowns 
button_create_input_table.on_click(functools.partial(create_input_table_manually,
                                                     dd_count_restr=dd_count_restr,
                                                     dd_count_x_vars=dd_count_x_vars,
                                                     dd_count_s_vars=dd_count_s_vars,
                                                     dd_count_a_vars=dd_count_a_vars,
                                                     display_table_input=display_table_input,
                                                     display_output=display_output,
                                                     M=M
                                                     )
                                   )

# observe Upload-Button for Upload
uploader.observe(functools.partial(create_input_table_with_file, display_table_input=display_table_input,
                                   display_output=display_output, M=M), names='value') 

# Branch and Bound

## Definiere Art/Anzahl der Variablen

Output()

Output()

Output()

In [6]:
9000 < np.inf

True